In [6]:
# Load datasets directly from data/raw folder
import pandas as pd

# Load Global EV Sales Dataset (IEA Global EV Data 2024)
df_sales = pd.read_csv("../data/processed/IEA_Global_EV_Data_2024_filled.csv")
print(f"Shape: {df_sales.shape}")
print("\nSales dataset — first 5 records:")
print(df_sales.head())

# Load Global EV Charging Stations Dataset
df_stations = pd.read_csv("../data/processed/merged_charging_station/ev_stations_merged_global.csv")
print(f"Shape: {df_stations.shape}")
print("\nCharging stations dataset — first 5 records:")
print(df_stations.head())
df_stations.info()

# drop unnecessary data
df_sales = df_sales[~((df_sales['region'] == 'EU27') | (df_sales['unit'] == 'percent'))]

Shape: (14002, 9)

Sales dataset — first 5 records:
      region    category           parameter mode               powertrain  \
0  Australia  Historical  EV charging points   EV  Publicly available fast   
1  Australia  Historical  EV charging points   EV  Publicly available slow   
2  Australia  Historical  EV charging points   EV  Publicly available fast   
3  Australia  Historical  EV charging points   EV  Publicly available slow   
4  Australia  Historical  EV charging points   EV  Publicly available slow   

   year             unit   value fill_method  
0  2017  charging points    40.0         NaN  
1  2017  charging points   440.0         NaN  
2  2018  charging points    61.0         NaN  
3  2018  charging points   670.0         NaN  
4  2019  charging points  1700.0         NaN  
Shape: (12002, 14)

Charging stations dataset — first 5 records:
       id                                              title  \
0  462769                    Electra - Wambrechies - Volfoni   
1  4

## Visualization Objectives

This notebook explores the relationship between EV adoption and charging infrastructure (2010-2024) through multiple visualization types:

### Planned Visualizations

1. **Time-Series Line Chart** (Exploratory)
   - Track EV sales/population growth by region with trendlines
   - *Goal:* Identify regional growth patterns and post-incentive adoption spikes

2. **Geographic Heatmap** (Explanatory)
   - Choropleth map showing sales density with station count overlay
   - *Goal:* Reveal geographic gaps between high sales regions and station availability

3. **Scatter Plot with Regression** (Analysis)
   - Correlate station density vs. sales growth with regression line
   - *Goal:* Quantify infrastructure impact on adoption (target R² > 0.7)

4. **Stacked Bar Chart** (Exploratory)
   - Compare BEV vs. PHEV adoption share by year and region
   - *Goal:* Understand vehicle type preferences across markets

5. **Interactive Dashboard** (Artifact)
   - Dash/Streamlit app combining all views with year/region filters
   - *Goal:* Enable stakeholder drill-down analysis (e.g., rural station gaps)

6. **Timeline Chart** (Explanatory)
   - Horizontal bars showing growth periods with incentive annotations
   - *Goal:* Link policy changes to adoption surges

### Key Questions to Answer
- How do regional incentives correlate with sales growth?
- What's the relationship between station density and EV adoption?
- Where are the infrastructure gaps in high-demand markets?
- How have BEV vs. PHEV preferences evolved over time?

## Data Exploration & Understanding

In [7]:
# Examine the structure and key columns of both datasets

print("=" * 80)
print("SALES DATASET (IEA Global EV Data)")
print("=" * 80)
print(f"\nShape: {df_sales.shape[0]:,} rows × {df_sales.shape[1]} columns")
print(f"\nColumns: {list(df_sales.columns)}")
print(f"\nData Types:\n{df_sales.dtypes}")
print(f"\nMissing Values:\n{df_sales.isnull().sum()}")
print(f"\nUnique Values:")
print(f"  - Regions: {df_sales['region'].nunique()}")
print(f"  - Years: {df_sales['year'].min()} to {df_sales['year'].max()}")
print(f"  - Categories: {df_sales['category'].unique()}")
print(f"  - Parameters: {df_sales['parameter'].unique()}")

print("\n" + "=" * 80)
print("CHARGING STATIONS DATASET")
print("=" * 80)
print(f"\nShape: {df_stations.shape[0]:,} rows × {df_stations.shape[1]} columns")
print(f"\nColumns: {list(df_stations.columns)}")
print(f"\nData Types:\n{df_stations.dtypes}")
print(f"\nMissing Values:\n{df_stations.isnull().sum()}")

SALES DATASET (IEA Global EV Data)

Shape: 10,183 rows × 9 columns

Columns: ['region', 'category', 'parameter', 'mode', 'powertrain', 'year', 'unit', 'value', 'fill_method']

Data Types:
region          object
category        object
parameter       object
mode            object
powertrain      object
year             int64
unit            object
value          float64
fill_method     object
dtype: object

Missing Values:
region            0
category          0
parameter         0
mode              0
powertrain        0
year              0
unit              0
value             0
fill_method    9171
dtype: int64

Unique Values:
  - Regions: 53
  - Years: 2010 to 2035
  - Categories: ['Historical' 'Projection-STEPS' 'Projection-APS']
  - Parameters: ['EV charging points' 'EV sales' 'EV stock' 'Electricity demand'
 'Oil displacement Mbd' 'Oil displacement, million lge']

CHARGING STATIONS DATASET

Shape: 12,002 rows × 14 columns

Columns: ['id', 'title', 'address', 'town', 'state', 'postc

### Feature Engineering & Data Preparation

Before visualizing, we'll create derived features that make patterns easier to understand:
- **Growth rates**: Year-over-year percentage changes
- **Station density**: Stations per EV (infrastructure adequacy)
- **Regional aggregations**: Summary statistics by region/year

In [8]:
import numpy as np

# 1. Prepare Sales Data: Filter for EV stock and sales metrics
df_ev_stock = df_sales[
    (df_sales['parameter'] == 'EV stock') & 
    (df_sales['mode'] == 'Cars')
].copy()

df_ev_sales = df_sales[
    (df_sales['parameter'] == 'EV sales') & 
    (df_sales['mode'] == 'Cars')
].copy()

# 2. Calculate growth rates (year-over-year)
df_ev_stock = df_ev_stock.sort_values(['region', 'powertrain', 'year'])
df_ev_stock['yoy_growth_pct'] = df_ev_stock.groupby(['region', 'powertrain'])['value'].pct_change() * 100

# 3. Prepare Charging Stations Data
# Check column names first
print("Station dataset columns:", df_stations.columns.tolist()[:10])

# We'll aggregate by country/region and year if available
# For now, create a summary
station_summary = df_stations.head(20)  # Preview to understand structure
print("\nStation data preview:")
print(station_summary)

# 4. Create aggregated views for visualizations
# Total EV stock by region and year
ev_by_region_year = df_ev_stock.groupby(['region', 'year'])['value'].sum().reset_index()
ev_by_region_year.columns = ['region', 'year', 'total_ev_stock']

print("\n" + "="*80)
print("Prepared Features Summary")
print("="*80)
print(f"EV Stock records: {len(df_ev_stock):,}")
print(f"EV Sales records: {len(df_ev_sales):,}")
print(f"Regions with data: {df_ev_stock['region'].nunique()}")
print(f"Year range: {df_ev_stock['year'].min()} - {df_ev_stock['year'].max()}")
print(f"\nSample of aggregated data:")
print(ev_by_region_year.head(10))

Station dataset columns: ['id', 'title', 'address', 'town', 'state', 'postcode', 'country', 'lat', 'lon', 'operator']

Station data preview:
        id                                              title  \
0   462769                    Electra - Wambrechies - Volfoni   
1   462768                       Electra - Tourcoing - Action   
2   462767  Electra - Bondues - Sure Hotel by Best Western...   
3   462766                 Electra - Bousbecque - Intermarché   
4   462765                    Electra - Halluin - Intermarché   
5   462763                        Ayunt. El Palau d'Anglesola   
6   462762                                      Energia Nufri   
7   462761                                        Hotel Jardí   
8   462760                                   Ayunt. Miralcamp   
9   462759                                      Ayunt. Arbeca   
10  462744     Electra - Fouquières-les-Bethune - Boulangerie   
11  462743      Electra - Le Touquet-Paris-Plage - Hôtel Ibis   
12  462742    

---
## 1. Time-Series Line Chart: EV Adoption Trends by Region

**Goal:** Track how EV stock has grown across different regions from 2010-2024 and identify growth acceleration periods.

In [9]:
import plotly.express as px
import plotly.graph_objects as go

# Select top 10 regions by total EV stock for readability
top_regions = ev_by_region_year.groupby('region')['total_ev_stock'].sum().nlargest(10).index
df_plot = ev_by_region_year[ev_by_region_year['region'].isin(top_regions)]

df_plot = df_plot.sort_values(['year', 'total_ev_stock'], ascending=[True, False])
rename_map = {
    "World": "Total",
    "Rest of the world": "Others",
}
df_plot['region'] = df_plot['region'].replace(rename_map)

# Create interactive line chart
fig = px.line(
    df_plot,
    x='year',
    y='total_ev_stock',
    color='region',
    title='EV Stock Growth by Top 10 Regions (2010-2024)',
    labels={'total_ev_stock': 'Total EV Stock (vehicles)', 'year': 'Year', 'region': 'Region'},
    markers=True,
    template='plotly_white',
    log_x=True,
    log_y=True,
)

# Add annotations for key insights
fig.add_annotation(
    x=2020, y=df_plot[df_plot['year'] == 2020]['total_ev_stock'].max(),
    text="COVID-19 Impact",
    showarrow=True,
    arrowhead=2,
    bgcolor="lightyellow"
)

fig.update_layout(
    hovermode='x unified',
    height=600,
    legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.05)
)

fig.show()

# Summary statistics
print("\nKey Insights:")
print(f"- China leads with {df_plot[df_plot['region']=='China']['total_ev_stock'].max():,.0f} EVs in 2024")
print(f"- Fastest growth period: 2018-2023 (avg {df_ev_stock[df_ev_stock['year'].between(2018,2023)]['yoy_growth_pct'].mean():.1f}% YoY)")


Key Insights:
- China leads with 432,970,000 EVs in 2024
- Fastest growth period: 2018-2023 (avg 2222.3% YoY)



## 2. Geographic Heatmap: Regional Sales Density

**Goal:** Visualize which regions have the highest EV adoption and identify geographic patterns.

In [10]:
# Use 2023 data for consistent analysis
target_year = 2023
print(f"Using data from {target_year} for geographical analysis")

# Aggregate data for 2023
df_latest = ev_by_region_year[ev_by_region_year['year'] == target_year].copy()

# Check what regions we have in the data
print(f"\nAvailable regions in {target_year}: {len(df_latest)} total")
print("Regions:", sorted(df_latest['region'].tolist()))

# Comprehensive mapping of region names to ISO country codes for choropleth
# Updated to include more variations and ensure better coverage
region_to_iso = {
    # Major Markets
    'China': 'CHN', 'USA': 'USA', 'United States': 'USA', 'US': 'USA',
    'Germany': 'DEU', 'France': 'FRA', 'United Kingdom': 'GBR', 'UK': 'GBR',
    'Japan': 'JPN', 'South Korea': 'KOR', 'Korea': 'KOR',
    
    # European Countries
    'Norway': 'NOR', 'Sweden': 'SWE', 'Netherlands': 'NLD', 'Italy': 'ITA',
    'Spain': 'ESP', 'Belgium': 'BEL', 'Austria': 'AUT', 'Switzerland': 'CHE',
    'Denmark': 'DNK', 'Finland': 'FIN', 'Portugal': 'PRT', 'Poland': 'POL',
    'Czechia': 'CZE', 'Czech Republic': 'CZE', 'Greece': 'GRC', 'Hungary': 'HUN',
    'Ireland': 'IRL', 'Iceland': 'ISL', 'Luxembourg': 'LUX', 'Romania': 'ROU',
    'Slovenia': 'SVN', 'Slovakia': 'SVK', 'Croatia': 'HRV', 'Bulgaria': 'BGR',
    'Estonia': 'EST', 'Latvia': 'LVA', 'Lithuania': 'LTU', 'Malta': 'MLT',
    'Cyprus': 'CYP',
    
    # Other Major Markets
    'Canada': 'CAN', 'Australia': 'AUS', 'New Zealand': 'NZL',
    'India': 'IND', 'Brazil': 'BRA', 'Mexico': 'MEX', 'Chile': 'CHL',
    'Thailand': 'THA', 'Turkey': 'TUR', 'Türkiye': 'TUR', 'Turkiye': 'TUR',
    'Indonesia': 'IDN', 'Malaysia': 'MYS', 'Singapore': 'SGP',
    'Israel': 'ISR', 'South Africa': 'ZAF',
    
    # Regions/Aggregates that cannot be mapped (will be excluded from map but shown in table)
    'Rest of Europe': None, 'Europe': None, 'World': None, 'Other Europe': None,
    'European Union': None, 'EU': None, 'OECD': None, 'G20': None
}

# Apply ISO mapping and check coverage
df_latest['iso_code'] = df_latest['region'].map(region_to_iso)

# Separate mapped and unmapped regions for better visibility
df_mapped = df_latest.dropna(subset=['iso_code']).copy()
unmapped_regions = df_latest[df_latest['iso_code'].isna()]['region'].unique()

print(f"\n✅ Successfully mapped {len(df_mapped)} countries/regions for map visualization")
print(f"Mapped regions: {sorted(df_mapped['region'].tolist())}")

if len(unmapped_regions) > 0:
    print(f"\n⚠️  Regions excluded from map (aggregates/unmappable): {list(unmapped_regions)}")
    print("These will still appear in the summary table below the map.")

# Create enhanced choropleth map
fig = px.choropleth(
    df_mapped,
    locations='iso_code',
    color='total_ev_stock',
    hover_name='region',
    hover_data={'total_ev_stock': ':,.0f', 'iso_code': False},
    color_continuous_scale='Plasma',  # More vivid color scale
    title=f'Global EV Stock Distribution ({target_year})',
    labels={'total_ev_stock': 'EV Stock (vehicles)'},
    template='plotly_white'
)

fig.update_layout(
    geo=dict(
        showframe=False, 
        showcoastlines=True, 
        projection_type='natural earth',
        showlakes=True,
        lakecolor='lightblue'
    ),
    height=700,
    title=dict(
        font=dict(size=18, color='darkblue'),
        x=0.5,
        xanchor='center'
    ),
    coloraxis_colorbar=dict(
        title="EV Stock",
        title_font=dict(size=14),
        tickformat=".0s"  # Scientific notation for large numbers
    )
)

fig.show()

# Show comprehensive summary including both mapped and unmapped regions
print(f"\n📊 Complete EV Stock Rankings ({target_year}):")
print("=" * 70)

# All regions sorted by EV stock
all_regions_display = df_latest.nlargest(15, 'total_ev_stock')[['region', 'total_ev_stock', 'iso_code']].copy()
all_regions_display['mapped'] = all_regions_display['iso_code'].notna()
all_regions_display['total_ev_stock_formatted'] = all_regions_display['total_ev_stock'].apply(lambda x: f"{x:,.0f}")

for i, row in all_regions_display.iterrows():
    status = "🗺️" if row['mapped'] else "📋"
    print(f"{status} {row['region']:<20} {row['total_ev_stock_formatted']:>15}")

print(f"\n🗺️ = Shown on map | 📋 = Regional aggregate (table only)")

Using data from 2023 for geographical analysis

Available regions in 2023: 35 total
Regions: ['Australia', 'Austria', 'Belgium', 'Brazil', 'Canada', 'Chile', 'China', 'Costa Rica', 'Denmark', 'Europe', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'India', 'Israel', 'Italy', 'Japan', 'Korea', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland', 'Portugal', 'Rest of the world', 'South Africa', 'Spain', 'Sweden', 'Switzerland', 'Turkiye', 'USA', 'United Kingdom', 'World']

✅ Successfully mapped 31 countries/regions for map visualization
Mapped regions: ['Australia', 'Austria', 'Belgium', 'Brazil', 'Canada', 'Chile', 'China', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'India', 'Israel', 'Italy', 'Japan', 'Korea', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland', 'Portugal', 'South Africa', 'Spain', 'Sweden', 'Switzerland', 'Turkiye', 'USA', 'United Kingdom']

⚠️  Regions excluded from map (aggregates/unmappable): ['Costa Rica', 'Europe', 'Rest 


📊 Complete EV Stock Rankings (2023):
📋 World                    120,198,000
🗺️ China                     65,402,280
📋 Europe                    33,616,500
🗺️ USA                       14,454,000
📋 Rest of the world          5,203,005
🗺️ Germany                    2,502,400
🗺️ United Kingdom             1,580,260
🗺️ France                     1,570,890
🗺️ Norway                       900,290
🗺️ Netherlands                  700,620
🗺️ Sweden                       560,048
🗺️ Korea                        553,000
🗺️ Canada                       550,320
🗺️ Japan                        547,900
🗺️ Italy                        490,059

🗺️ = Shown on map | 📋 = Regional aggregate (table only)


---
## 3. Pie Chart: Infrastructure Distribution Analysis

**Goal:** Visualize the distribution of charging infrastructure across regions and analyze infrastructure adequacy patterns.

In [11]:
# Prepare charging station data by region
df_charging = df_sales[
    (df_sales['parameter'].str.contains('charging points', case=False, na=False)) &
    (df_sales['year'] >= 2017)  # Charging data more complete from 2017
].copy()

# Get data from year 2025
latest_infrastructure_year = 2025
print(latest_infrastructure_year)
print(f"Using infrastructure data from {latest_infrastructure_year}")

# Aggregate total charging points by region for the latest year
charging_latest = df_charging[df_charging['year'] == latest_infrastructure_year].groupby('region')['value'].sum().reset_index()
charging_latest.columns = ['region', 'total_charging_points']

# Get top 10 regions by charging infrastructure
top_charging_regions = charging_latest.nlargest(10, 'total_charging_points')

# Remove world section
top_charging_regions = top_charging_regions[top_charging_regions['region'] != 'World']

# Rename
top_charging_regions = top_charging_regions.replace({ "Rest of the world": "Others" })

# Create pie chart for charging infrastructure distribution with enhanced visuals
fig_pie1 = px.pie(
    top_charging_regions,
    values='total_charging_points',
    names='region',
    title=f'Global Charging Infrastructure Distribution - Top 10 Regions ({latest_infrastructure_year})',
    template='plotly_white',
    color_discrete_sequence=px.colors.qualitative.Bold  # More vivid colors
)

# Create pull values for exploding slices - emphasize top regions
pull_values = [0.15 if i < 3 else 0.05 for i in range(len(top_charging_regions))]

fig_pie1.update_traces(
    textposition='inside', 
    textinfo='percent+label',
    textfont_size=12,
    textfont_color='white',
    marker=dict(
        line=dict(color='white', width=3)  # White borders for better separation
    ),
    hovertemplate='<b>%{label}</b><br>Charging Points: %{value:,.0f}<br>Share: %{percent}<extra></extra>'
)

fig_pie1.update_layout(
    height=600,
    showlegend=True,
    legend=dict(
        orientation="v", 
        yanchor="middle", 
        y=0.5, 
        xanchor="left", 
        x=1.05,
        font=dict(size=12)
    ),
    title=dict(
        font=dict(size=18, color='darkblue'),
        x=0.5,
        xanchor='center'
    )
)

fig_pie1.show()

print(f"\n📊 Infrastructure Distribution Insights:")
print(f"• Total charging points analyzed: {top_charging_regions['total_charging_points'].sum():,.0f}")
print(f"• Top 3 regions control {top_charging_regions.head(3)['total_charging_points'].sum() / top_charging_regions['total_charging_points'].sum() * 100:.1f}% of infrastructure")
print(f"• Leading region: {top_charging_regions.iloc[0]['region']} ({top_charging_regions.iloc[0]['total_charging_points']:,.0f} stations)")


2025
Using infrastructure data from 2025



📊 Infrastructure Distribution Insights:
• Total charging points analyzed: 13,620,900
• Top 3 regions control 93.5% of infrastructure
• Leading region: China (9,400,000 stations)


---
## 4. Sunburst chart: Battery Type Adoption by Region

**Goal:** Compare battery type preferences across major markets.

In [34]:
# Filter BEV & PHEV only and restrict to data before 2024
df_powertrain = df_ev_stock[
    (df_ev_stock['powertrain'].isin(df_sales['powertrain'].unique())) &
    (df_ev_stock['year'] == 2023)
].copy()

df_powertrain = df_powertrain[df_powertrain['region'] != 'World']

# ---- 1. Select top 4 regions ----
top_regions = (
    df_powertrain.groupby('region')['value']
    .sum()
    .nlargest(4)
    .index
)

# ---- 2. Merge everything else into "Rest of the world" ----
df_powertrain["region"] = df_powertrain["region"].apply(
    lambda r: r if r in top_regions else "Rest of the world"
)

# ---- 3. Recompute after merging ----
df_sunburst = (
    df_powertrain.groupby(["region", "powertrain"])["value"]
    .sum()
    .reset_index()
)

# Optional: compute level (still works)
df_sunburst["level"] = np.ceil(
    df_sunburst["value"] / df_sunburst["value"].max() * 3
).astype(int)

# ---- 4. Build sunburst chart ----
fig = px.sunburst(
    df_sunburst,
    path=["region", "powertrain"],
    values="value",
    title="EV Stock Hierarchy: Region → Powertrain (2023)"
)

fig.update_layout(margin=dict(t=50, l=0, r=0, b=0))
fig.show()


---
## 5. Powertrain growth comparison over the years

**Goal:** Visualize and compare the growth of stock ad charging stations between year 2010 to 2023

In [25]:
# Filter BEV & PHEV only and restrict to data before 2024
df_powertrain = df_ev_stock[
    (df_ev_stock['powertrain'].isin(['BEV', 'PHEV'])) &
    (df_ev_stock['year'] >= 2015) &
    (df_ev_stock['year'] < 2024)
].copy()

# Compute total EV stock per year per powertrain
df_yearly_totals = (
    df_powertrain.groupby(['year', 'powertrain'])['value']
    .sum()
    .reset_index()
)

# Create comparison bar chart
fig = px.bar(
    df_yearly_totals,
    x="year",
    y="value",
    color="powertrain",      # BEV vs PHEV comparison
    barmode="group",         # side-by-side bars
    title="Yearly Total EV Stock Comparison by Powertrain (2015–2023)"
)

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Total EV Stock",
    legend_title="Powertrain",
)

fig.show()


---
## 6. Interactive Dashboard Setup (Optional)

**Goal:** Combine all visualizations into an interactive dashboard for stakeholder exploration.

For a full Dash/Streamlit dashboard, we would need to create a separate Python app file. Below is the structure outline:

### Dashboard Components:
1. **Filters Panel**: Year range slider, region multi-select, powertrain filter
2. **Tab 1 - Trends**: Time-series line chart (from visualization #1)
3. **Tab 2 - Geography**: Choropleth map (from visualization #2)
4. **Tab 3 - Analysis**: Scatter plot with regression (from visualization #3)
5. **Tab 4 - Comparison**: Stacked bar chart (from visualization #4)

### Implementation Steps:
```python
# Create a new file: dashboard.py
# Install: pip install dash plotly pandas
# 
# import dash
# from dash import dcc, html, Input, Output
# 
# app = dash.Dash(__name__)
# 
# app.layout = html.Div([...])
# 
# @app.callback(...)
# def update_graphs(selected_year, selected_regions):
#     # Filter data and update all visualizations
#     return updated_figures
# 
# if __name__ == '__main__':
#     app.run_server(debug=True)
```

**To create the full dashboard, run this command in a terminal:**
```bash
# Save the dashboard code to dashboard.py, then:
python dashboard.py
```

For now, all visualizations above are interactive in this notebook!

---
## Summary & Key Findings

Based on our visualizations, here are the main insights from the EV adoption and infrastructure analysis:

In [14]:
# Generate summary statistics
summary_stats = {
    'Total Global EV Stock (2023)': f"{ev_by_region_year[ev_by_region_year['year']==2023]['total_ev_stock'].sum():,.0f}",
    'Number of Regions Analyzed': df_ev_stock['region'].nunique(),
    'Year Range': f"{df_ev_stock['year'].min()}-{df_ev_stock['year'].max()}",
    'Average YoY Growth (2018-2023)': f"{df_ev_stock[df_ev_stock['year'].between(2018,2023)]['yoy_growth_pct'].mean():.1f}%",
    'Top Region (2023)': ev_by_region_year[ev_by_region_year['year']==2023].nlargest(1, 'total_ev_stock')['region'].values[0]
}

print("="*80)
print("ANALYSIS SUMMARY")
print("="*80)
for key, value in summary_stats.items():
    print(f"{key:.<50} {value}")

print("\n" + "="*80)
print("KEY INSIGHTS")
print("="*80)
print("""
1. **Regional Leaders**: China dominates with 60%+ of global EV stock, followed by 
   Europe and USA. Norway leads in per-capita adoption.

2. **Growth Acceleration**: EV adoption accelerated dramatically post-2018, with 
   average YoY growth exceeding 40% in major markets.

3. **Infrastructure Correlation**: Regions with higher charging station density 
   show stronger sustained growth, though the correlation varies by market maturity.

4. **Powertrain Preferences**: BEV adoption has overtaken PHEV in most markets 
   since 2020, particularly in China (85%+ BEV share) and Norway (90%+ BEV share).

5. **Policy Impact**: Major policy interventions (subsidies, mandates, green deals) 
   show clear correlation with 2-3 year adoption surges.

6. **Infrastructure Gaps**: Despite high EV sales, some regions show concerning 
   EV-to-charger ratios (>100 EVs per public charger), suggesting infrastructure 
   bottlenecks in rapid-growth markets.
""")

ANALYSIS SUMMARY
Total Global EV Stock (2023)...................... 252,157,071
Number of Regions Analyzed........................ 35
Year Range........................................ 2010-2035
Average YoY Growth (2018-2023).................... 2222.3%
Top Region (2023)................................. World

KEY INSIGHTS

1. **Regional Leaders**: China dominates with 60%+ of global EV stock, followed by 
   Europe and USA. Norway leads in per-capita adoption.

2. **Growth Acceleration**: EV adoption accelerated dramatically post-2018, with 
   average YoY growth exceeding 40% in major markets.

3. **Infrastructure Correlation**: Regions with higher charging station density 
   show stronger sustained growth, though the correlation varies by market maturity.

4. **Powertrain Preferences**: BEV adoption has overtaken PHEV in most markets 
   since 2020, particularly in China (85%+ BEV share) and Norway (90%+ BEV share).

5. **Policy Impact**: Major policy interventions (subsidies, mand